In [4]:
import seaborn as sns
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import japanize_matplotlib
import warnings
warnings.filterwarnings('ignore')

#データの読み込み
#公開データはExcel形式なので、read_excel関数で直接読み込む
df = pd.read_excel('https://archive.ics.uci.edu/ml/machine-learning-databases/00352/Online%20Retail.xlsx')

#項目名を日本語に変更
columns = [
    '発注番号', '商品番号', '商品説明', '商品個数', '明細書発行日',
    '商品単価', '顧客番号', '国名'
]
df.columns = columns

In [5]:
#データ件数確認
print(df.shape[0])

#データ内容確認
display(df.head())

541909


,発注番号,商品番号,商品説明,商品個数,明細書発行日,商品単価,顧客番号,国名
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


In [6]:
#欠損値確認
print(df.isnull().sum())

発注番号           0
商品番号           0
商品説明        1454
商品個数           0
明細書発行日         0
商品単価           0
顧客番号      135080
国名             0
dtype: int64


In [7]:
#国名確認
print(df['国名'].value_counts().head(10))

United Kingdom    495478
Germany             9495
France              8557
EIRE                8196
Spain               2533
Netherlands         2371
Belgium             2069
Switzerland         2002
Portugal            1519
Australia           1259
Name: 国名, dtype: int64


In [10]:
#「発注種別」の追加

#全処理用にデータをコピー
df2 = df.copy()

#発注番号の頭１桁を別項目に抽出
#(5:新規オーダー C:キャンセル)
df2['発注種別'] = df2['発注番号'].map(lambda x: str(x)[0])

#結果確認
display(df2.head())

#種別個数確認
print(df2['発注種別'].value_counts())

,発注番号,商品番号,商品説明,商品個数,明細書発行日,商品単価,顧客番号,国名,発注種別
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom,5
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,5
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom,5
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,5
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,5


5    532618
C      9288
A         3
Name: 発注種別, dtype: int64


In [11]:
#新規オーダーのみ抽出
df2 = df2[df2['発注種別'] == '5']

#件数確認
print(df2.shape[0])

532618


In [12]:
#分析対象をフランスに限定する
df3 = df2[df2['国名']=='France']

#件数確認
print(df3.shape[0])

8408


In [13]:
#発注番号と商品番号をキーに商品個数を集計する
w1 = df3.groupby(['発注番号', '商品番号'])['商品個数'].sum()

#結果確認
display(w1.head())

発注番号    商品番号 
536370  10002    48
        21035    18
        21724    12
        21731    24
        21791    24
Name: 商品個数, dtype: int64

In [16]:
#商品番号を列に移動(unstack関数の利用)
w2 = w1.unstack().reset_index().fillna(0).set_index('発注番号')

#サイズ確認
print(w2.shape)

#結果確認
display(w2.head())

(392, 1542)


商品番号,10002,10120,10125,10135,11001,15036,15039,16012,16048,16218,...,85232D,90030B,90030C,90184B,90184C,90201B,90201C,C2,M,POST
発注番号,,,,,,,,,,,,,,,,,,,,,
536370,48.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0
536852,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
536974,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0
537065,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9.0
537463,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0


In [17]:
#集計結果が正か0かでTrue/Falseを設定
basket_df = w2.apply(lambda x: x>0)

#結果確認
display(basket_df.head())

商品番号,10002,10120,10125,10135,11001,15036,15039,16012,16048,16218,...,85232D,90030B,90030C,90184B,90184C,90201B,90201C,C2,M,POST
発注番号,,,,,,,,,,,,,,,,,,,,,
536370,True,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True
536852,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True
536974,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True
537065,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True
537463,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True


In [19]:
#ライブラリ(mlxtend)の読み込み
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

In [20]:
#アプリオリによる分析
freq_items1 = apriori(basket_df, min_support = 0.06,
                      use_colnames = True)

#結果確認
display(freq_items1.sort_values('support',
            ascending = False).head(10))

#itemset 数確認
print(freq_items1.shape[0])

,support,itemsets
61,0.765306,(POST)
52,0.188776,(23084)
14,0.181122,(21731)
37,0.170918,(22554)
39,0.168367,(22556)
114,0.165816,"(POST, 23084)"
24,0.158163,(22326)
82,0.158163,"(POST, 21731)"
4,0.153061,(20725)
89,0.147959,"(POST, 22326)"


134


支持度を算出して、閾値が0.06より大きいものを抽出している

In [21]:
#アソシエーションルールの抽出
a_rules1 = association_rules(freq_items1, metric = 'lift',
             min_threshold = 1)

#リフト値でソート
a_rules1 = a_rules1.sort_values('lift',
        ascending = False).reset_index(drop=True)

#結果確認
display(a_rules1.head(10))
#ルール確認
print(a_rules1.shape[0])

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(23254),(23256),0.071429,0.068878,0.063776,0.892857,12.962963,0.058856,8.690476
1,(23256),(23254),0.068878,0.071429,0.063776,0.925926,12.962963,0.058856,12.535714
2,(22727),"(22728, 22726)",0.094388,0.073980,0.063776,0.675676,9.133271,0.056793,2.855230
3,"(22728, 22726)",(22727),0.073980,0.094388,0.063776,0.862069,9.133271,0.056793,6.565689
4,(22727),"(POST, 22726)",0.094388,0.084184,0.071429,0.756757,8.989353,0.063483,3.765023
5,"(POST, 22726)",(22727),0.084184,0.094388,0.071429,0.848485,8.989353,0.063483,5.977041
6,(22726),"(22728, 22727)",0.096939,0.073980,0.063776,0.657895,8.892922,0.056604,2.706829
7,"(22728, 22727)",(22726),0.073980,0.096939,0.063776,0.862069,8.892922,0.056604,6.547194
8,(22727),(22726),0.094388,0.096939,0.079082,0.837838,8.642959,0.069932,5.568878
9,(22726),(22727),0.096939,0.094388,0.079082,0.815789,8.642959,0.069932,4.916181


206


In [22]:
#チューニング
#アプリオリによる分析
freq_items2 = apriori(basket_df, min_support = 0.065,
                      use_colnames = True)

#アソシエーションルールの抽出
a_rules2 = association_rules(freq_items2, metric = 'lift',
             min_threshold = 1)

#リフト値でソート
a_rules2 = a_rules2.sort_values('lift',
        ascending = False).reset_index(drop=True)

#結果確認
display(a_rules2.head(10))

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(22727),"(POST, 22726)",0.094388,0.084184,0.071429,0.756757,8.989353,0.063483,3.765023
1,"(POST, 22726)",(22727),0.084184,0.094388,0.071429,0.848485,8.989353,0.063483,5.977041
2,(22727),(22726),0.094388,0.096939,0.079082,0.837838,8.642959,0.069932,5.568878
3,(22726),(22727),0.096939,0.094388,0.079082,0.815789,8.642959,0.069932,4.916181
4,(22726),"(POST, 22727)",0.096939,0.086735,0.071429,0.736842,8.495356,0.063021,3.470408
5,"(POST, 22727)",(22726),0.086735,0.096939,0.071429,0.823529,8.495356,0.063021,5.117347
6,"(22728, POST)",(22727),0.089286,0.094388,0.066327,0.742857,7.870270,0.057899,3.521825
7,(22727),"(22728, POST)",0.094388,0.089286,0.066327,0.702703,7.870270,0.057899,3.063312
8,(22728),(22727),0.102041,0.094388,0.073980,0.725000,7.681081,0.064348,3.293135
9,(22727),(22728),0.094388,0.102041,0.073980,0.783784,7.681081,0.064348,4.153061


In [23]:
#調査対象の集合
t_set = set([23254, 23256])

#1回目の分析freq_items1から該当行を抽出
idx1 = freq_items1['itemsets'].map(
    lambda x: not x.isdisjoint(t_set))
item1 = freq_items1[idx1]

#2回目の分析freq_items2から該当行を抽出
idx2 = freq_items2['itemsets'].map(
    lambda x: not x.isdisjoint(t_set))
item2 = freq_items2[idx2]

#結果確認
display(item1)
display(item2)

,support,itemsets
58,0.071429,(23254)
59,0.068878,(23256)
118,0.063776,"(23256, 23254)"


,support,itemsets
53,0.071429,(23254)
54,0.068878,(23256)


二つの出力結果を比較すると、min_supportの値を0.06から0.065に変更したことで、
(23254 AND 23256)の条件の支持度が含まれなくなったことが分かる。